#### 20260109 update to intrinsics summaries from calib.db

In [89]:
import json
import pandas as pd
import glob
import os
import numpy as np

#### set up the infrastructure to read calib.db json output

In [90]:
json_dir = r'C:\CJH\python\FRC\vision\2025\python_2025_multicam_2429\intrinsics'

In [91]:
def get_calibration_df(directory_path):
    """
    Scans a directory for .json calibration files and returns a 
    formatted Pandas DataFrame of intrinsic parameters.
    """
    pattern = os.path.join(directory_path, "**", "*.json")
    json_files = glob.glob(pattern, recursive=True)
    
    all_data = []
    for file in json_files:
        with open(file, 'r') as f:
            data = json.load(f)
            
            # Flatten the camera matrix and resolution into top-level keys
            matrix = data['camera_matrix']
            dist = data['distortion_coefficients']
            
            flat_entry = {
                'camera': data.get('camera'),
                'fx': matrix[0][0],
                'fy': matrix[1][1],
                'cx': matrix[0][2],
                'cy': matrix[1][2],
                'd1': dist[0], 'd2': dist[1], 'd3': dist[2], 'd4': dist[3], 'd5': dist[4],
                'x_res': data['img_size'][0],
                'y_res': data['img_size'][1],
                'avg_reprojection_error': data.get('avg_reprojection_error')
            }
            all_data.append(flat_entry)
            
    return pd.DataFrame(all_data)

In [92]:
# this is not averaged 
df = get_calibration_df(json_dir)
df

,camera,fx,fy,cx,cy,d1,d2,d3,d4,d5,x_res,y_res,avg_reprojection_error
0,Arducam OV9281 USB Camera (0c45:6366) D,914.460343,913.301293,640.416995,431.817370,0.040270,-0.079934,-0.001447,-0.001467,0.017368,1280,720,0.151267
1,Arducam OV9281 USB Camera (0c45:6366) C,911.755241,910.998429,639.565341,355.667149,0.015835,0.012237,-0.000311,-0.000309,-0.071728,1280,720,0.184136
2,Arducam OV9281 USB Camera (0c45:6366) D,914.496388,913.775728,638.130482,433.823280,0.018852,0.010766,-0.000114,-0.001031,-0.066830,1280,720,0.212522
3,Arducam OV9281 USB Camera (0c45:6366) C,905.020096,904.908192,641.530919,355.331567,0.040589,-0.084570,-0.001092,0.000162,0.027033,1280,720,0.174544
4,Arducam OV9281 USB Camera (0c45:6366) D,906.850812,906.548036,640.528639,430.352100,0.031969,-0.039090,-0.000676,-0.001431,-0.017724,1280,720,0.204831
5,Arducam OV9281 USB Camera (0c45:6366) C,903.302048,903.883499,640.597384,351.716775,0.022264,-0.004489,-0.002316,-0.000507,-0.036895,1280,720,0.225927
6,Arducam OV9281 USB Camera (0c45:6366) D,913.571845,913.215550,641.891040,431.636329,0.041439,-0.069300,-0.000460,-0.000496,0.007992,1280,720,0.153964
7,Arducam OV9281 USB Camera (0c45:6366) C,909.833058,909.186827,641.899906,353.012725,0.022803,-0.014974,-0.001200,0.000263,-0.043698,1280,720,0.188269
8,Arducam OV9281 USB Camera (0c45:6366) A,922.677198,922.960043,649.254674,354.815012,0.051186,-0.069687,-0.001384,0.000382,0.014409,1280,720,0.111758
9,Arducam OV9281 USB Camera (0c45:6366) A,915.997699,915.759586,651.026905,356.870558,0.052011,-0.068721,0.000133,0.000031,0.010439,1280,720,0.111441


---
####  now average the camera data in case there are outliers

In [93]:
def get_model_averages(df):
    """Returns the mean intrinsic values grouped by camera model."""
    return df.groupby("camera").mean(numeric_only=True).reset_index()

In [94]:
averages = get_model_averages(df)
averages

,camera,fx,fy,cx,cy,d1,d2,d3,d4,d5,x_res,y_res,avg_reprojection_error
0,Arducam OV9281 USB Camera (0c45:6366) A,919.355796,919.668696,649.473605,354.142917,0.049019,-0.055785,-0.001200,-0.000336,-0.001505,1280.0,720.0,0.106459
1,Arducam OV9281 USB Camera (0c45:6366) B,919.480682,919.667128,631.660167,321.310451,0.055867,-0.070836,0.000018,-0.000227,0.003206,1280.0,720.0,0.134209
2,Arducam OV9281 USB Camera (0c45:6366) C,907.477611,907.244237,640.898388,353.932054,0.025373,-0.022949,-0.001230,-0.000098,-0.031322,1280.0,720.0,0.193219
3,Arducam OV9281 USB Camera (0c45:6366) D,912.344847,911.710152,640.241789,431.907270,0.033132,-0.044389,-0.000674,-0.001106,-0.014799,1280.0,720.0,0.180646
4,HD Pro Webcam C920 (046d:082d) A,925.850180,927.242728,641.539322,350.240760,0.122302,-0.239596,-0.001272,-0.001263,0.100909,1280.0,720.0,0.322754
5,HD Pro Webcam C920 (046d:082d) A 202601,913.515787,915.986792,640.281698,348.040271,0.088603,-0.214448,-0.000540,0.000288,0.186985,1280.0,720.0,0.240440
6,HD Pro Webcam C920 (046d:08e5) D,968.355862,967.505072,641.909914,354.769446,0.043553,-0.176323,-0.001455,0.000244,0.146537,1280.0,720.0,0.297501
7,HD Pro Webcam C920 (046d:08e5) D 202601,936.917446,937.506090,640.684801,357.183256,0.015043,-0.091083,-0.000595,-0.001451,0.029925,1280.0,720.0,0.189540
8,c922 Pro Stream Webcam (046d:085c) B,968.285652,968.173786,654.426110,346.705363,0.055570,-0.172193,-0.001258,0.001099,0.116279,1280.0,720.0,0.336494
9,c922 Pro Stream Webcam (046d:085c) B 202601,945.646767,945.874717,642.947819,351.470947,0.029382,-0.126729,-0.000625,-0.000047,0.065257,1280.0,720.0,0.225767


In [95]:
# Filter for any camera string containing the C920 hardware ID
c920_all = df[df['camera'].str.contains('046d:082d', case=False)]
c920_all

,camera,fx,fy,cx,cy,d1,d2,d3,d4,d5,x_res,y_res,avg_reprojection_error
25,HD Pro Webcam C920 (046d:082d) A 202601,898.181799,899.711831,636.556028,344.426866,0.096452,-0.355196,-0.001015,-0.002880,0.635439,1280,720,0.237300
26,HD Pro Webcam C920 (046d:082d) A 202601,930.932027,931.992715,632.826604,341.552318,0.081861,-0.154831,-0.001252,0.000950,0.009107,1280,720,0.231037
27,HD Pro Webcam C920 (046d:082d) A 202601,907.161037,913.086115,645.156418,354.043916,0.070777,-0.123778,0.000351,0.000663,0.020975,1280,720,0.293065
28,HD Pro Webcam C920 (046d:082d) A 202601,917.788286,919.156506,646.587741,352.137984,0.105322,-0.223987,-0.000243,0.002417,0.082421,1280,720,0.200359
29,HD Pro Webcam C920 (046d:082d) A,928.852996,929.375144,633.646223,339.706460,0.120686,-0.259233,-0.002912,-0.004044,0.109042,1280,720,0.173346
30,HD Pro Webcam C920 (046d:082d) A,899.578722,908.384113,629.636624,379.920347,0.084391,0.017028,0.003278,-0.006039,-0.227201,1280,720,0.303580
31,HD Pro Webcam C920 (046d:082d) A,943.544533,942.210821,650.125409,354.435591,0.134863,-0.301446,0.003089,0.000299,0.174244,1280,720,0.417804
32,HD Pro Webcam C920 (046d:082d) A,930.052512,929.162610,659.538920,334.364162,0.178170,-0.509721,-0.009057,0.005655,0.457351,1280,720,0.547980
33,HD Pro Webcam C920 (046d:082d) A,920.982305,922.828709,637.633857,350.026848,0.111764,-0.212518,-0.001473,-0.002481,0.060641,1280,720,0.238594
34,HD Pro Webcam C920 (046d:082d) A,932.090013,931.494972,638.654902,342.991152,0.103937,-0.171686,-0.000555,-0.000971,0.031377,1280,720,0.255219


In [96]:
for _, row in averages.iterrows():
    # Construct the dictionary for unscaled intrinsics
    intrinsics_dict = {
        "fx": round(row['fx'], 2),
        "fy": round(row['fy'], 2),
        "cx": round(row['cx'], 2),
        "cy": round(row['cy'], 2)
    }
    
    # Extract unscaled distortion coefficients into a list
    dist_list = [
        round(row['d1'], 8), 
        round(row['d2'], 8), 
        round(row['d3'], 8), 
        round(row['d4'], 8), 
        round(row['d5'], 8)
    ]
    
    # Print with the camera name preamble and distortions on the next line
    print(f"{row['camera']} :")
    print(f"    \"intrinsics\": {json.dumps(intrinsics_dict)},")
    print(f"    \"distortions\": {dist_list},")

Arducam OV9281 USB Camera (0c45:6366) A :
    "intrinsics": {"fx": 919.36, "fy": 919.67, "cx": 649.47, "cy": 354.14},
    "distortions": [0.04901888, -0.05578499, -0.00119951, -0.00033551, -0.00150501],
Arducam OV9281 USB Camera (0c45:6366) B :
    "intrinsics": {"fx": 919.48, "fy": 919.67, "cx": 631.66, "cy": 321.31},
    "distortions": [0.05586731, -0.07083556, 1.843e-05, -0.00022748, 0.00320574],
Arducam OV9281 USB Camera (0c45:6366) C :
    "intrinsics": {"fx": 907.48, "fy": 907.24, "cx": 640.9, "cy": 353.93},
    "distortions": [0.02537283, -0.02294888, -0.00122956, -9.774e-05, -0.03132185],
Arducam OV9281 USB Camera (0c45:6366) D :
    "intrinsics": {"fx": 912.34, "fy": 911.71, "cx": 640.24, "cy": 431.91},
    "distortions": [0.0331324, -0.04438929, -0.00067446, -0.00110617, -0.0147985],
HD Pro Webcam C920 (046d:082d) A :
    "intrinsics": {"fx": 925.85, "fy": 927.24, "cx": 641.54, "cy": 350.24},
    "distortions": [0.12230188, -0.23959588, -0.00127158, -0.00126347, 0.10090911],


In [97]:
# target resolution for the scaled output
target_resolution = (640, 360)

for _, row in averages.iterrows():
    # Scale the average data for this camera
    scaled = scale_intrinsics(target_resolution, row)
    
    # Format the intrinsics dictionary
    intrinsics_dict = {
        "fx": scaled['fx'],
        "fy": scaled['fy'],
        "cx": scaled['cx'],
        "cy": scaled['cy']
    }
    
    # Extract distortion coefficients into a list
    dist_list = [
        round(row['d1'], 8), 
        round(row['d2'], 8), 
        round(row['d3'], 8), 
        round(row['d4'], 8), 
        round(row['d5'], 8)
    ]
    
    # Print with the camera name preamble and distortions on the next line
    print(f"{row['camera']} (640x360) :")
    print(f"    \"intrinsics\": {json.dumps(intrinsics_dict)},")
    print(f"    \"distortions\": {dist_list},")

Arducam OV9281 USB Camera (0c45:6366) A (640x360) :
    "intrinsics": {"fx": 459.678, "fy": 459.834, "cx": 324.74, "cy": 177.07},
    "distortions": [0.04901888, -0.05578499, -0.00119951, -0.00033551, -0.00150501],
Arducam OV9281 USB Camera (0c45:6366) B (640x360) :
    "intrinsics": {"fx": 459.74, "fy": 459.834, "cx": 315.83, "cy": 160.66},
    "distortions": [0.05586731, -0.07083556, 1.843e-05, -0.00022748, 0.00320574],
Arducam OV9281 USB Camera (0c45:6366) C (640x360) :
    "intrinsics": {"fx": 453.739, "fy": 453.622, "cx": 320.45, "cy": 176.97},
    "distortions": [0.02537283, -0.02294888, -0.00122956, -9.774e-05, -0.03132185],
Arducam OV9281 USB Camera (0c45:6366) D (640x360) :
    "intrinsics": {"fx": 456.172, "fy": 455.855, "cx": 320.12, "cy": 215.95},
    "distortions": [0.0331324, -0.04438929, -0.00067446, -0.00110617, -0.0147985],
HD Pro Webcam C920 (046d:082d) A (640x360) :
    "intrinsics": {"fx": 462.925, "fy": 463.621, "cx": 320.77, "cy": 175.12},
    "distortions": [0.12

---
#### tools to scale down if we choose to

In [98]:
def scale_intrinsics(target_res, source_data, use_max_focal=True):
    """
    Scales intrinsic parameters from a source resolution to a target resolution.
    
    Args:
        target_res (tuple): (width, height) e.g., (640, 360)
        source_data (dict/Series): Dictionary containing fx, fy, cx, cy, x_res, y_res
        use_max_focal (bool): If True, uses the larger scale factor for focal length 
                              (standard for maintaining FOV during cropping).
    """
    target_x, target_y = target_res
    source_x, source_y = source_data['x_res'], source_data['y_res']
    
    scale_x = target_x / source_x
    scale_y = target_y / source_y
    
    # Focal length scaling
    f_scale = max(scale_x, scale_y) if use_max_focal else (scale_x, scale_y)
    
    if isinstance(f_scale, tuple):
        fx = source_data['fx'] * f_scale[0]
        fy = source_data['fy'] * f_scale[1]
    else:
        fx = source_data['fx'] * f_scale
        fy = source_data['fy'] * f_scale

    # Principal point scaling
    cx = source_data['cx'] * scale_x
    cy = source_data['cy'] * scale_y
    
    # Calculate FOV for verification
    fov_h = 2 * np.degrees(np.arctan(target_x / (2 * fx)))
    fov_v = 2 * np.degrees(np.arctan(target_y / (2 * fy)))
    
    return {
        'fx': round(fx, 3), 'fy': round(fy, 3),
        'cx': round(cx, 2), 'cy': round(cy, 2),
        'fov_h': round(fov_h, 1), 'fov_v': round(fov_v, 1)
    }

---
#### show min/max on each parameter

In [99]:
# Assuming 'df' is the DataFrame returned by get_calibration_df()
intrinsics_cols = ['fx', 'fy', 'cx', 'cy', 'avg_reprojection_error']

# Group by the camera model and calculate min, max, and mean for each parameter
summary_table = df.groupby('camera')[intrinsics_cols].agg(['min', 'max', 'mean'])

# Optional: Clean up the column names for better readability
summary_table.columns = ['_'.join(col).strip() for col in summary_table.columns.values]

# Display the result
summary_table.reset_index()

,camera,fx_min,fx_max,fx_mean,fy_min,fy_max,fy_mean,cx_min,cx_max,cx_mean,cy_min,cy_max,cy_mean,avg_reprojection_error_min,avg_reprojection_error_max,avg_reprojection_error_mean
0,Arducam OV9281 USB Camera (0c45:6366) A,915.997699,922.677198,919.355796,915.759586,922.960043,919.668696,648.542023,651.026905,649.473605,351.629074,356.870558,354.142917,0.084695,0.117942,0.106459
1,Arducam OV9281 USB Camera (0c45:6366) B,917.609991,922.111195,919.480682,917.789436,922.376487,919.667128,629.409153,634.105079,631.660167,319.204682,323.638127,321.310451,0.119979,0.146790,0.134209
2,Arducam OV9281 USB Camera (0c45:6366) C,903.302048,911.755241,907.477611,903.883499,910.998429,907.244237,639.565341,641.899906,640.898388,351.716775,355.667149,353.932054,0.174544,0.225927,0.193219
3,Arducam OV9281 USB Camera (0c45:6366) D,906.850812,914.496388,912.344847,906.548036,913.775728,911.710152,638.130482,641.891040,640.241789,430.352100,433.823280,431.907270,0.151267,0.212522,0.180646
4,HD Pro Webcam C920 (046d:082d) A,899.578722,943.544533,925.850180,908.384113,942.210821,927.242728,629.636624,659.538920,641.539322,334.364162,379.920347,350.240760,0.173346,0.547980,0.322754
5,HD Pro Webcam C920 (046d:082d) A 202601,898.181799,930.932027,913.515787,899.711831,931.992715,915.986792,632.826604,646.587741,640.281698,341.552318,354.043916,348.040271,0.200359,0.293065,0.240440
6,HD Pro Webcam C920 (046d:08e5) D,946.534460,997.738542,968.355862,949.427351,998.418380,967.505072,609.115581,657.192224,641.909914,347.132442,364.959054,354.769446,0.170899,0.407191,0.297501
7,HD Pro Webcam C920 (046d:08e5) D 202601,934.240514,939.594377,936.917446,934.511796,940.500384,937.506090,637.953667,643.415935,640.684801,354.460820,359.905692,357.183256,0.171084,0.207996,0.189540
8,c922 Pro Stream Webcam (046d:085c) B,950.404937,995.372669,968.285652,949.202323,992.396066,968.173786,639.569603,677.113390,654.426110,323.240883,367.940903,346.705363,0.261106,0.403546,0.336494
9,c922 Pro Stream Webcam (046d:085c) B 202601,939.640912,952.971292,945.646767,940.988463,950.868057,945.874717,638.866572,644.910484,642.947819,348.491346,357.857016,351.470947,0.195423,0.280497,0.225767
